In [12]:
import __future__
import sys, os
import json
from Bio import SeqIO
# Assume that nextstrain/augur directory is next to h3n2_reassortment directory
sys.path.append('../../../hutch/nextstrain/augur/')
from base import io_util

In [13]:
def load_features(reference, feature_names=None):
    # This function was taken directly from nextstrain/augur/augur/uitils.py#216
    #read in appropriately whether GFF or Genbank
    #checks explicitly for GFF otherwise assumes Genbank
    if not os.path.isfile(reference):
        print("ERROR: reference sequence not found. looking for", reference)
        return None

    features = {}
    if '.gff' in reference.lower():
        #looks for 'gene' and 'gene' as best for TB
        try:
            from BCBio import GFF #Package name is confusing - tell user exactly what they need!
        except ImportError:
            print("ERROR: Package BCBio.GFF not found! Please install using \'pip install bcbio-gff\' before re-running.")
            return None
        limit_info = dict( gff_type = ['gene'] )

        with open(reference) as in_handle:
            for rec in GFF.parse(in_handle, limit_info=limit_info):
                for feat in rec.features:
                    if feature_names is not None: #check both tags; user may have used either
                        if "gene" in feat.qualifiers and feat.qualifiers["gene"][0] in feature_names:
                            fname = feat.qualifiers["gene"][0]
                        elif "locus_tag" in feat.qualifiers and feat.qualifiers["locus_tag"][0] in feature_names:
                            fname = feat.qualifiers["locus_tag"][0]
                        else:
                            fname = None
                    else:
                        if "gene" in feat.qualifiers:
                            fname = feat.qualifiers["gene"][0]
                        else:
                            fname = feat.qualifiers["locus_tag"][0]
                    if fname:
                        features[fname] = feat

            if feature_names is not None:
                for fe in feature_names:
                    if fe not in features:
                        print("Couldn't find gene {} in GFF or GenBank file".format(fe))

    else:
        from Bio import SeqIO
        for feat in SeqIO.read(reference, 'genbank').features:
            if feat.type=='CDS':
                if "locus_tag" in feat.qualifiers:
                    fname = feat.qualifiers["locus_tag"][0]
                    if feature_names is None or fname in feature_names:
                        features[fname] = feat
                elif "gene" in feat.qualifiers:
                    fname = feat.qualifiers["gene"][0]
                    if feature_names is None or fname in feature_names:
                        features[fname] = feat
            elif feat.type=='source': #read 'nuc' as well for annotations - need start/end of whole!
                features['nuc'] = feat

    return features

In [14]:
data_file = "../hyphy/aligned_fastas/HA_aligned.fasta"
ha_ref = "../src/config/reference_h3n2_ha.gb"
output_file = "../hyphy/aligned_fastas/HA1.trimmed.test.fasta"

features = load_features(ha_ref, "HA1")

In [15]:
sequences = []
with open(data_file, "rU") as handle:
    for record in SeqIO.parse(handle, "fasta"):
        seq = record.seq
        record.seq = features['HA1'].extract(seq)
        sequences.append(record)

In [16]:
with open(output_file, "w") as output_handle:
    SeqIO.write(sequences, output_handle, "fasta")